In [12]:
from datasets import load_dataset


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer


model_name = "kanhatakeyama/0405_100m_clean_ja"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [16]:
from datasets import DatasetDict


dataset = load_dataset('hatakeyama-llm-team/WikiBookJa', split="train")
# データセットを訓練データと検証データに分割
train_test_split = dataset.train_test_split(test_size=0.1)  # 10%を検証データに
dataset = DatasetDict({
    'train': train_test_split['train'],
    'validation': train_test_split['test']
})
def tokenize_and_pad(examples):
    # テキストをトークナイズ
    result = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
    return result

# データセットにトークナイズとパディングを適用
tokenized_datasets = dataset.map(tokenize_and_pad, batched=True)

Map: 100%|██████████| 1633/1633 [00:01<00:00, 1621.19 examples/s]


In [17]:
from peft import LoraConfig, get_peft_model
r=8
lora_alpha=r
bit=16
target_modules= [
    #"lm_head",
    "q_proj",
    "k_proj",
    "v_proj",
    #"o_proj",
    #"gate",
    #"w1",
    #"w2",
    #"w3"
]
target_modules=[
    "c_attn",
    "c_proj",
]
peft_config = LoraConfig(
    task_type="CAUSAL_LM", inference_mode=False, r=r, lora_alpha=lora_alpha,
    lora_dropout=0.1,
    target_modules=target_modules,
)
model = get_peft_model(model, peft_config)

In [18]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=1,
    gradient_accumulation_steps=32,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

trainer.train()


ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.